In [1]:
from pathlib import Path

output_dir = Path("./output")
redirect_links_dir = Path(output_dir / "redirect_links").resolve()  # Resolve to absolute path
# """Get all non-temporary pickle files in the directory."""
        
        
pickle_files = [
    f for f in redirect_links_dir.glob("*.pickle")
    if not f.name.endswith("_temp.pickle")
]

pickle_files

[PosixPath('/mnt/resources2/weekly-projects/scrapy_engine_v3/server/crawled_data/output/redirect_links/redirect_links_d4ffb8bb0c5915b218014856d301b209_20250112_161251_191255.pickle'),
 PosixPath('/mnt/resources2/weekly-projects/scrapy_engine_v3/server/crawled_data/output/redirect_links/redirect_links_270eea41476d716ba2660f23dd1d1280_20250112_161253_292935.pickle')]

In [7]:
import pickle
from bloom import get_bloom_thread

crawled_bloom = get_bloom_thread(save_file='crawled_bloom_filter.pkl', scalable=True)
error_bloom = get_bloom_thread(save_file='error_bloom_filter.pkl', scalable=True)

# all_links=set()
links_to_crawl = []

for pickle_file in pickle_files:
    redirect_links = []
    with open(pickle_file,'rb') as file:
        redirect_links = pickle.load(file)
        
    for link in redirect_links:
        # all_links.add(link)
        if link not in crawled_bloom and link not in error_bloom:
            crawled_bloom.add(link)
            error_bloom.add(link)
            links_to_crawl.append(link)

print(f'to_crawl:{len(links_to_crawl)}')
# print(f'all:{len(all_links)}')


		 Creaetd new bloom filter

		 Creaetd new bloom filter

to_crawl:71


In [4]:
crawled_bloom.save()
error_bloom.save()

Bloom filter saved to file: crawled_bloom_filter.pkl

Bloom filter saved to file: error_bloom_filter.pkl



In [ ]:
def load_urls_from_pickle(self):
    """
    Load redirect URLs from pickle files into MongoDB after filtering through bloom filters.
    
    The function:
    1. Reads redirect URLs from pickle files in the redirect_links directory
    2. Filters URLs using bloom filters to avoid duplicates
    3. Adds new URLs to MongoDB with metadata
    4. Updates and saves bloom filters
    5. Cleans up processed pickle files
    
    Returns:
        int: Number of new URLs added to MongoDB
    """
    # Get all non-temporary pickle files
    pickle_files = list(self.redirect_links_dir.glob("[!_]*[!_temp].pickle"))
    
    if not pickle_files:
        self.logger.info("No pickle files found to process")
        return 0
    
    links_to_crawl = set()  # Using set for better performance
    
    try:
        # Process each pickle file
        for pickle_path in pickle_files:
            try:
                with pickle_path.open('rb') as file:
                    redirect_links = pickle.load(file)
                    
                    # Filter links through bloom filters
                    new_links = {
                        link for link in redirect_links
                        if link not in self.crawled_bloom 
                        and link not in self.error_bloom
                    }
                    
                    # Update bloom filters with new links
                    for link in new_links:
                        self.crawled_bloom.add(link)
                        self.error_bloom.add(link)
                    
                    links_to_crawl.update(new_links)
                    
            except (IOError, pickle.UnpicklingError) as e:
                self.logger.error(f"Failed to process {pickle_path}: {str(e)}")
                continue
        
        # Prepare documents for MongoDB
        if links_to_crawl:
            current_time = time.time()
            docs = [
                {
                    "url": link,
                    "status": "to_crawl",
                    "timestamp": current_time,
                    "crawling_count": 0
                } for link in links_to_crawl
            ]
            
            # Batch insert into MongoDB
            try:
                result = self.collection.insert_many(docs, ordered=False)
                inserted_count = len(result.inserted_ids)
                self.logger.info(f"Added {inserted_count} new URLs to MongoDB")
            except Exception as e:
                self.logger.error(f"Failed to insert documents to MongoDB: {str(e)}")
                # Still save bloom filters even if MongoDB insert fails
                raise
            
            # Save bloom filters after successful MongoDB insertion
            try:
                self.crawled_bloom.save()
                self.error_bloom.save()
                self.logger.info("Bloom filters updated and saved")
            except Exception as e:
                self.logger.error(f"Failed to save bloom filters: {str(e)}")
                raise
        
        # Clean up processed pickle files
        for pickle_path in pickle_files:
            try:
                pickle_path.unlink()
            except OSError as e:
                self.logger.error(f"Failed to remove {pickle_path}: {str(e)}")
        
        return len(links_to_crawl)
        
    except Exception as e:
        self.logger.error(f"Unexpected error in load_urls_from_pickle: {str(e)}")
        raise

In [9]:
a=set()
[a.add(i) for i in ['a','b','c']]

b=['d','e','f']
a.update(b)
a

{'a', 'b', 'c', 'd', 'e', 'f'}